In [10]:
import pandas as pd
import redis
import csv

In [85]:
import numpy as np

In [3]:
df

,state,state_po,county_str,office,special,candidate,party,writein,candidatevotes,won,MOV16,MOV18,16to18swing
0,Arizona,AZ,apache_az,US Senator,False,Angela Green,green,False,961.0,False,0.368776,0.338343,-0.030434
1,Arizona,AZ,apache_az,US Senator,False,Kyrsten Sinema,democrat,False,16298.0,True,0.368776,0.338343,-0.030434
2,Arizona,AZ,apache_az,US Senator,False,Martha McSally,republican,False,7810.0,False,0.368776,0.338343,-0.030434
3,Arizona,AZ,apache_az,US Senator,False,NaN,NaN,True,18.0,False,0.368776,0.338343,-0.030434
4,Arizona,AZ,cochise_az,US Senator,False,Angela Green,green,False,1212.0,False,-0.225856,-0.209577,0.016279
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9104,Wyoming,WY,washakie_wy,US Senator,False,Write-Ins,NaN,True,6.0,False,-0.640377,-0.590032,0.050345
9105,Wyoming,WY,weston_wy,US Senator,False,Gary Trauner,democrat,False,321.0,False,-0.781044,-0.726935,0.054109
9106,Wyoming,WY,weston_wy,US Senator,False,John Barrasso,republican,False,2275.0,True,-0.781044,-0.726935,0.054109
9107,Wyoming,WY,weston_wy,US Senator,False,Joseph Porambo,libertarian,False,89.0,False,-0.781044,-0.726935,0.054109


In [114]:
df = pd.read_csv('updatedcleandata3.csv')
df_json = df.to_json(orient='records')
df.head()

,state,state_po,county_str,office,special,candidate,party,writein,candidatevotes,won,MOV16,MOV18,16to18swing
0,Arizona,AZ,apache_az,US Senator,False,Angela Green,green,False,961.0,False,0.368776,0.338343,-0.030434
1,Arizona,AZ,apache_az,US Senator,False,Kyrsten Sinema,democrat,False,16298.0,True,0.368776,0.338343,-0.030434
2,Arizona,AZ,apache_az,US Senator,False,Martha McSally,republican,False,7810.0,False,0.368776,0.338343,-0.030434
3,Arizona,AZ,apache_az,US Senator,False,NaN,NaN,True,18.0,False,0.368776,0.338343,-0.030434
4,Arizona,AZ,cochise_az,US Senator,False,Angela Green,green,False,1212.0,False,-0.225856,-0.209577,0.016279


In [112]:
r.flushdb()

True

In [18]:
df = df[df['writein']==False]

In [23]:
df.to_csv('updatedcleandata2.csv',index=False)

In [113]:
redis_host = 'localhost'
redis_port = 6379

r = redis.Redis(host=redis_host, port=redis_port,db=10)

In [6]:
r.set('cleandata', df_json)
#stored_df_json = r.get('my_dataframe')
#stored_df = pd.read_json(stored_df_json)

#print(stored_df)

True

In [115]:
with open('updatedcleandata.csv', newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        key = f"{row['candidate']}:{row['county_str']}"
        r.hset(key, mapping=row)

In [116]:
r.hget('Angela Green:apache_az','16to18swing')

b'-0.03043389389716611'

In [27]:
#r.get('cleandata')

In [117]:

column_to_query = '16to18swing'
min_value = -0.15

# Scan all keys in Redis
cursor = 0
matched_keys = []

while True:
    # Perform scan operation
    cursor, keys = r.scan(cursor, match='*:*')

    # Iterate over the keys and check the column value
    for key in keys:
        # Get the column value for the key
        value = r.hget(key, column_to_query)

        # Check if the value is greater than the desired value
        if value and float(value) < min_value:
            matched_keys.append(key)
            
        #print(key)

    # Exit the loop if the scan is complete
    if cursor == 0:
        break

# Retrieve the values for the matched keys
matched_values = []
for key in matched_keys:
    # Retrieve all key-value pairs for the key
    values = r.hgetall(key)

    # Decode the byte strings to regular strings
    decoded_values = {k.decode('utf-8'): v.decode('utf-8') for k, v in values.items()}

    matched_values.append(decoded_values)

# Print the matched values
#for value in matched_values:
#    print(value)

# Close the Redis connection
r.close()


In [118]:
state_d = dict()
cand_d = dict()
county_d = dict()

for value in matched_values:
    if value['candidate'] in cand_d:
        cand_d[value['candidate']] = cand_d[value['candidate']] + 1
    else:
        cand_d[value['candidate']] = 1
        
    if value['state'] in state_d:
        state_d[value['state']] = state_d[value['state']] + 1
    else:
        state_d[value['state']] = 1
        
    if value['county_str'] in county_d:
        county_d[value['county_str']] = county_d[value['county_str']] + 1
    else:
        county_d[value['county_str']] = 1

In [119]:
sorted(state_d.values())

[9, 11, 12, 30]

In [120]:
dict(sorted(state_d.items(), key=lambda item: item[1], reverse=True))

{'Utah': 30, 'Mississippi': 12, 'Missouri': 11, 'Texas': 9}

In [86]:
np.abs(-5)

5

In [121]:
df[(df['state_po']=='MN') & (df['16to18swing'] < 0)]

,state,state_po,county_str,office,special,candidate,party,writein,candidatevotes,won,MOV16,MOV18,16to18swing
